# **Trích xuất các thuộc tính khác từ mô tả bài đăng với mô hình LLM**

Mô hình ngôn ngữ lớn là một loại thuật toán trí tuệ nhân tạo áp dụng các kỹ thuật mạng thần kinh với nhiều tham số để xử lý và hiểu ngôn ngữ hoặc văn bản của con người bằng kỹ thuật học tự giám sát. 

LLM API  là sự tương tác kỹ thuật với các hệ thống AI phức tạp có khả năng xử lý, hiểu và tạo ra ngôn ngữ của con người. Các API này hoạt động như một kênh giữa các thuật toán phức tạp về hiệu suất LLM và các ứng dụng khác nhau, cho phép tích hợp liền mạch các chức năng xử lý ngôn ngữ vào các giải pháp phần mềm.

**Mục đích** \
Sử dụng mô hình LLM để trích xuất các thuộc tính sau từ nội dung mô tả của bài đăng:
* is_real_estate_post -> Nhằm phân biệt với các bài viết spam, mặc định None
* area_LLM -> diện tích, mặc định None
* bedroom_LLM -> số phòng ngủ, mặc định None
* wc_LLM -> số phòng vệ sinh, mặc định None
* count_conveniences -> số tiện ích xung quanh, mặc định None
* alleyway_property -> trong hẻm, mặc định là None
* under_mortgage -> có đang bị thế chấp không, mặc định None -> thể hiện sự ownership
* residential_purpose -> Chắc chắn nằm trên đất thổ cư, mặc định None
* furnished -> bán nhà có bán kèm nột thất không, mặc định None
* has_lowerlevel -> tầng hầm, default None
* has_rooftop -> sân thượng, mặc định None
* numbers_of_floors -> số tầng, không tính tầng trệt, sân thượng, mặc định 0
* street_name -> tên đường, mặc định None

**Prompt**
Xem file prompt_for_extracting trong thư mục crawl_data

In [ ]:
# Sử dụng LLM API trên AnyScale web
#import re
def crawl_data(text_required):
    with requests.Session() as s:
        # thử lại 5 lần nếu truy cập vào web bị lỗi ,backoff_factor quy định thời gian chờ
        retries = Retry(total=5, backoff_factor=0.1, status_forcelist=[500, 502, 503, 504])
        s.mount("https://", HTTPAdapter(max_retries=retries))
        api_base = "https://api.endpoints.anyscale.com/v1"
        token = "esecret_c3bs9hkq93mbayfhaqht59qd1m"
        url = f"{api_base}/chat/completions"
        body = {
            "model": "meta-llama/Meta-Llama-3-70B-Instruct",
            "messages": [{
              "role": "system",
              "content": ("You are a data scientist, and I want to extract useful data from a description in a Vietnamese real estate post. I’ll provide you with that description, and you must only return the keys and values in JSON format without any introduction. The response mustn't contain any explanation. The keys contain those properties: is_real_estate_post, area_LLM, bedroom_LLM, total_room_LLm, wc_LLM, count_conveniences, alleyway_property, under_mortgage, residential_purpose, furnished, has_lowerlevel, has_rooftop, numbers_of_floors, street_name. is_real_estate_post is boolean value (1/0), default is null. area_LLM is a float number, converted into m2 unit, greater than 0, default is null. bedroom_LLM is an integer number, greater than 0, default is null. wc_LLM is an integer number, greater than 0, default is null. count_conveniences is an integer number, greater or equal 0, default is 0. The list of convenience-related words is defined as:a = ('school', 'hospital', 'supermarket', 'shopping center', 'market', 'mall', 'mart').For each word from this list that appears in the description, increment the count_conveniences by 1.If none of the words from the list are found in the description, then the value of count_conveniences remains unchanged Alleyway_property is boolean value (1/0), default is null. under_mortgage is boolean value (1/0), default is null. residential_purpose is boolean value (1/0), default value is null, it ensures the buyer have full ownership rights for residential purposes. has_basement, has_rooftop, furnished are boolean values (1/0), default is null. numbers_of_floors is an integer number, greater or equal 0, default is 0, doesn't include the ground floor and rooftoop floor. street_name is a string, default is null, it represents the name of the street where the property is located in Ho Chi Minh City, Viet Name. If the description doesn't contain enough information to determine the value of a property, the corresponding value in the dictionary should be set to default value.")
            },
                {"role": "user", "content": f"{text_required}"}],
            "temperature": 0.2, # kiểm soát sự ngẫu nhiên của output
            "max_tokens": 256,
            "top_p": 1,   # kiểm soát tính đa dạng của đầu ra
            "frequency_penalty": 0 # kiểm soát sự lặp từ
        }
        headers = {"Authorization": f"Bearer {token}","Content-Type": "application/json"}
        try:
            response = s.post(url, headers=headers, json=body)
            response.raise_for_status() # kiểm tra xem yêu cầu có được chấp thuận không
            data = response.json()  # chuyển sang dictionary
            if "choices" in data and len(data["choices"]) > 0: # kiểm tra xem dữ liệu có tồn tại hay không
                return re.sub('\n', '',data["choices"][0]["message"]["content"]) # trả về lời nhắn đầu tiên
            else:
                return "{}"
        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}") # xuất ra lỗi
            return "{}"

In [ ]:
def ensure_key_present(text, key):
    if f"'{key}'" not in text and f'"{key}"' not in text:
        text = re.sub(r'(\{)', rf'\1\n    "{key}": {0},', text)
    return text

In [ ]:
# xử lý chuỗi bằng regrex

def remove_fault(text):
    # xử lý chuỗi trước và sau dấu ngoặc nhọn
    text = re.sub(r'^.*?{', '{', text, flags=re.DOTALL)
    text = re.sub(r'}.*$', '}', text, flags=re.DOTALL)
    # xử lý khi 1 chuỗi trả về giá None, thay vào đó là giá trị 0
    none_pattern = r'None'
    cleaned_text = re.sub(none_pattern, '0', text)
    # xử lý khi bị lỗi trả về float("NaN"),thay vào đó là giá trị null
    cleaned_text = re.sub(r'float\("NaN"\)', 'null', cleaned_text)
    # xử lý khi trả '', thay vào đó là ""
    cleaned_text = re.sub(r"'([^']*)'", r'"\1"', cleaned_text)
    # xử lý khi chuỗi có xuống dòng
    cleaned_text = re.sub(r'\n', '', cleaned_text)
    # xử lý khi LLM không trả về đủ các key
    key=['is_real_estate_post', 'area_LLM', 'bedroom_LLM', 'wc_LLM', 'count_conveniences', 'alleyway_property', 'under_mortgage', 'residential_purpose', 'numbers_of_floors' ,'has_rooftop', 'furnished','street_name']
    for k in key:
        cleaned_text = ensure_key_present(cleaned_text, k)
    # xử lý khi LLM trả về có dấu phẩy ở cuối
    pattern = re.compile(r',\s*([}\]])')
    cleaned_text = pattern.sub(r'\1', cleaned_text)

    return cleaned_text

In [ ]:
def process_text(input):
    text_required, article_id = input
    output = crawl_data(text_required)
    output = remove_fault(output)
    # Kiểm tra tính hợp lệ dạng JSON
    try:
        result = json.loads(output)
        result['article_id'] = article_id
    except (json.JSONDecodeError, TypeError) as e:
        print(f"Error decoding JSON for index {0}: {e}")
        print(output)
        return None
    return result

In [ ]:
import multiprocessing as mp
import json
# Text nhập vào, việc mô tả hợp lí, rõ ràng giúp cho câu trả lời có chất lượng tốt hơn
result_list = []
inputs = []
for i in range(raw_data.shape[0]):
    text_required = "This is the description in Vietnamese:"
    text_required += str(raw_data['title'][i]) + ' ' + str(raw_data['description'][i]) + '.'
    inputs.append((text_required, raw_data['article_id'][i]))

# Xử lý đa luồng với số luồng là 20, giúp tăng tốc độ, với cách xử lí này các câu trả lời không bị đảo thứ tự cho nhau, mà vẫn thêm đúng như thứ tự đã nhập vào ban đầu.
with mp.Pool(processes=20) as pool:
    results = pool.map(process_text, inputs)
# Tăng tốc độ xử lý ,Tăng khả năng đa nhiệm: Multiprocessing cho phép thực hiện nhiều tác vụ đồng thời, làm tăng khả năng đa nhiệm của ứng dụng, phân phối công việc: bạn có thể phân chia công việc thành các tiến trình riêng biệt, giúp tận dụng tối đa tài nguyên hệ thống., Isolation: Mỗi tiến trình trong multiprocessing có môi trường thực thi riêng của nó, giúp tránh tình trạng xung đột dữ liệu và lỗi do tài nguyên được chia sẻ không an toàn.

# Thêm kết quả vào mảng
result_df = pd.DataFrame(results)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
result_df.to_csv('/content/drive/My Drive/raw_data_3_extracted_by_LLM.csv', index=False)